In [10]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio.v3 as iio

from sklearn.preprocessing import LabelEncoder

import torch
import torchdata.datapipes as dp
from torch.utils.data import DataLoader

import torchvision
torchvision.disable_beta_transforms_warning();

import torchvision.transforms.v2 as t
import torchmetrics
import pytorch_lightning as pl

from tqdm import tqdm
from typing import Callable, Any

from dotenv import load_dotenv
load_dotenv();

from hyperparameters import Hyperparameters
from datamodules import ImagenetteDataLoader, viz_batch 


In [2]:
IMAGENETTE = Path.home() / "datasets" / "imagenette" / "imagenette2"

In [27]:
class ClassificationModel(pl.LightningModule):
    def __init__(self, model, params: Hyperparameters):
        super().__init__()
        self.model = model
        self.params = params
        self.model = model
        
        self._set_metrics()
        self.save_hyperparameters(
            {i:params.get_dict()[i] for i in params.get_dict().keys() if i!='criterion'},
            ignore = ["model"]
        ) 
    
    def forward(self, batch):
        x, _ = batch
        return self.model(x)

    def _set_metrics(self):
        self.train_metrics = torchmetrics.Accuracy("multiclass", 
                                    num_classes=self.params.num_classes,
                                    average = "micro")
        self.test_metrics = torchmetrics.Accuracy("multiclass",
                                    num_classes=self.params.num_classes, 
                                    average = "micro")

    def training_step(self, batch, batch_idx):
        loss = self._forward_pass(batch, self.train_metrics)
        self.log("train_loss", loss, on_step=True, on_epoch = True)
        self.log("train_acc", self.train_metrics, on_epoch=True, on_step=False)
        return loss

    def test_step(self, batch, batch_idx):
        loss = self._forward_pass(batch, self.test_metrics)
        self.log("test_loss", loss)
        self.log("test_acc", self.test_metrics, on_epoch=True, on_step=False)
    
    #def validation_step(self, batch, batch_idx):
        #loss = self._forward_pass(batch, self.val_metrics)
        #self.log("val_loss", loss)
      
    def configure_optimizers(self):
        return self.params.optimizer(self.model.parameters(), 
                                     lr = self.params.learning_rate)

    def _forward_pass(self, batch, metrics):
        x, y = batch
        y_pred = self.model(x)
        metrics(y_pred, y) 
        return self.params.criterion(y_pred, y)


In [28]:
experiment = Hyperparameters(
    task = "multiclass_classification",
    random_seed = 42,
    num_classes = 10,
    test_split = .3,
    metrics = ["accuracy", "f1score"],

    learning_rate = 1e-6,
    batch_size =  64,
    num_workers = 16,
    optimizer = torch.optim.Adam,
    criterion = torch.nn.CrossEntropyLoss(),

    local_cache_limit=0
)
pl.seed_everything(experiment.random_seed);

Global seed set to 42


In [29]:
imagenette_dl = ImagenetteDataLoader(IMAGENETTE, experiment)
alexnet = torchvision.models.AlexNet(experiment.num_classes)
classifier = ClassificationModel(alexnet, experiment)

In [32]:
trainer = pl.Trainer(
    overfit_batches = 1.0,
    logger = True,
    max_epochs = 50 
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1.0)` was configured so 100% of the batches will be used..


In [33]:
trainer.fit(model = classifier, datamodule = imagenette_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | AlexNet            | 57.0 M
1 | train_metrics | MulticlassAccuracy | 0     
2 | test_metrics  | MulticlassAccuracy | 0     
-----------------------------------------------------
57.0 M    Trainable params
0         Non-trainable params
57.0 M    Total params
228.179   Total estimated model params size (MB)


Epoch 42:  32%|███▏      | 48/148 [00:59<02:03,  1.23s/it, v_num=5] 